# LLM-powered AI Agents

Table of contents
1. Understanding LLMs
2. Tools
3. Chat-based AI Agents
4. Service-based AI agents

In [1]:
from language_models.proxy_client import BTPProxyClient
from language_models.settings import settings

proxy_client = BTPProxyClient(
    client_id=settings.CLIENT_ID,
    client_secret=settings.CLIENT_SECRET,
    auth_url=settings.AUTH_URL,
    api_base=settings.API_BASE,
)

## 1. Understanding LLMs

In [2]:
from language_models.models.llm import OpenAILanguageModel, ChatMessage, ChatMessageRole

In [3]:
llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model="gpt-35-turbo",
    max_tokens=256,
    temperature=0.0,
)

In [4]:
prompt = """Take the following movie review and determine the sentiment of the review.

Movie review:
Wow! This movie was incredible. The acting was superb, and
the plot kept me on the edge of my seat. I highly recommend it!
"""

response = llm.get_completion([ChatMessage(role=ChatMessageRole.USER, content=prompt)])
print(response)

Sentiment: Positive


In [5]:
prompt = """Take the following movie review and determine the sentiment of the review.

Movie review:
Wow! This movie was incredible. The acting was superb, and
the plot kept me on the edge of my seat. I highly recommend it!

Respond with positive or negative.
"""

response = llm.get_completion([ChatMessage(role=ChatMessageRole.USER, content=prompt)])
print(response)

positive


In [6]:
system_prompt = "Take the following movie review and determine the sentiment of the review. Respond with 1 (positive) or 0 (negative)."

prompt = "Wow! This movie was incredible. The acting was superb, and the plot kept me on the edge of my seat. I highly recommend it!"

response = llm.get_completion([
    ChatMessage(role=ChatMessageRole.SYSTEM, content=system_prompt), 
    ChatMessage(role=ChatMessageRole.USER, content=prompt),
])
print(response)

1


In [7]:
system_prompt = "Take the following movie review determine the sentiment of the review. Respond with 1 (positive) or 0 (negative)."

prompt = "Will it rain in Seattle today?"

response = llm.get_completion([
    ChatMessage(role=ChatMessageRole.SYSTEM, content=system_prompt), 
    ChatMessage(role=ChatMessageRole.USER, content=prompt),
])
print(response)

I'm sorry, I am an AI language model and I do not have access to real-time weather information. I recommend checking a reliable weather website or using a weather app to get the most accurate and up-to-date forecast for Seattle.


In [8]:
system_prompt = """Take the following movie review and determine the sentiment of the review. 

Respond with 1 (positive) or 0 (negative).

If you don't receive a movie review, respond with -1.
"""

prompt = "Will it rain in Seattle today?"

response = llm.get_completion([
    ChatMessage(role=ChatMessageRole.SYSTEM, content=system_prompt), 
    ChatMessage(role=ChatMessageRole.USER, content=prompt),
])
print(response)

-1


## 2. Tools

In [9]:
import json
from language_models.tools.tool import Tool
from pydantic import BaseModel, Field
from typing import Any

In [10]:
prompt = "Total Raw Cost = $549.72 + $6.98 + $41.00 + $35.00 + $552.00 + $76.16 + $29.12" # answer: $1,289.98

response = llm.get_completion([ChatMessage(role=ChatMessageRole.USER, content=prompt)])
print(response)

Total Raw Cost = $1,290.98


In [11]:
def calculator(expression: str) -> Any:
    return eval(expression)

class Calculator(BaseModel):
    expression: str = Field(description="A math expression.")

calculator_tool = Tool(
    func=calculator,
    name="Calculator",
    description="Use this tool when you want to do calculations.",
    args_schema=Calculator
)
print(calculator_tool)

tool name: Calculator, tool description: Use this tool when you want to do calculations., tool input: {{'expression': {{'description': 'A math expression.', 'title': 'Expression', 'type': 'string'}}}}


In [12]:
system_prompt = f"""Take the following prompt and calculate the result.

Respond to the user as helpfully and accurately as possible. You have access to the following tools:
{calculator_tool}

Use a json blob to specify a tool by providing an action (tool name) and an action_input (tool input).

Always use the following JSON response format:
{{
    "thought": You should always think about what to do consider previous and subsequent steps,
    "tool": The tool to use,
    "tool_input": A valid dictionary in this format {{"<key>": <value>, ...}},
}}
"""

prompt = "Total Raw Cost = $549.72 + $6.98 + $41.00 + $35.00 + $552.00 + $76.16 + $29.12"

response = llm.get_completion([
    ChatMessage(role=ChatMessageRole.SYSTEM, content=system_prompt), 
    ChatMessage(role=ChatMessageRole.USER, content=prompt),
])
response = json.loads(response, strict=False)
print(json.dumps(response, indent=4))

{
    "thought": "To calculate the total raw cost, you need to add up all the individual costs.",
    "tool": "Calculator",
    "tool_input": {
        "expression": "549.72 + 6.98 + 41.00 + 35.00 + 552.00 + 76.16 + 29.12"
    }
}


In [13]:
print(calculator(**response["tool_input"]))

1289.98


In [14]:
system_prompt = f"""Take the following prompt and calculate the result.

Respond to the user as helpfully and accurately as possible. You have access to the following tools:
{calculator_tool}

Use a json blob to specify a tool by providing an action (tool name) and an action_input (tool input).

Always use the following JSON response format:
{{
    "thought": You should always think about what to do consider previous and subsequent steps,
    "tool": The tool to use,
    "tool_input": A valid dictionary in this format {{"<key>": <value>, ...}},
}}
... (this Thought/Action/Observation can repeat N times)
When you know the final answer, use the following JSON response format:
{{
    "thought": I know the final answer,
    "tool": Final Answer,
    "tool_input": The final answer to the question,
}}
"""

prompt = "Total Raw Cost = $549.72 + $6.98 + $41.00 + $35.00 + $552.00 + $76.16 + $29.12"

response = llm.get_completion([
    ChatMessage(role=ChatMessageRole.SYSTEM, content=system_prompt), 
    ChatMessage(role=ChatMessageRole.USER, content=prompt),
    ChatMessage(role=ChatMessageRole.ASSISTANT, content=json.dumps(response)),
    ChatMessage(role=ChatMessageRole.ASSISTANT, content=f"Response of Calculator tool: {calculator(**response['tool_input'])}"),
])
response = json.loads(response, strict=False)
print(json.dumps(response, indent=4))

{
    "thought": "I know the final answer",
    "tool": "Final Answer",
    "tool_input": 1289.98
}


## 3. Chat-based AI Agents

In [15]:
from language_models.agents.react import ReActAgent
from language_models.tools.earthquake import earthquake_tools
from language_models.tools.current_date import current_date_tool

In [16]:
system_prompt = """You are an United States Geological Survey expert who can answer questions regarding earthquakes and can run forecasts.

Use the current date tool to access the local date and time before using other tools.

Take the following question and answer it as accurately as possible.
"""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=1024,
    float=0.0,
)

class Output(BaseModel):
    content: str = Field(description="The final answer.")

agent = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt="{question}",
    task_prompt_variables=["question"],
    tools=earthquake_tools + [current_date_tool],
    output_format=Output,
    iterations=5,
)

In [17]:
response = agent.invoke({"question": "How many earthquakes have occurred for the past week with a magnitude of 5 or greater?"})

03/05/24 16:25:28 INFO Thought: First, I need to get the current date to calculate the start date for the past week.
03/05/24 16:25:28 INFO Tool: Current Date
03/05/24 16:25:28 INFO Tool input: {}
03/05/24 16:25:28 INFO Tool response: 2024-05-03 16:25:28.250997
03/05/24 16:25:38 INFO Thought: Now that I have the current date, I can calculate the start date for the past week. Then, I will use the 'Count' tool to find out how many earthquakes with a magnitude of 5 or greater have occurred during this period.
03/05/24 16:25:38 INFO Tool: Count
03/05/24 16:25:38 INFO Tool input: {'start_time': '2024-04-26T16:25:28.250997', 'min_magnitude': 5}
03/05/24 16:25:39 INFO Tool response: {'count': 32, 'maxAllowed': 20000}
03/05/24 16:25:47 INFO Thought: I now know the final answer
03/05/24 16:25:47 INFO Tool: Final Answer
03/05/24 16:25:47 INFO Tool input: {'content': 'There have been 32 earthquakes with a magnitude of 5 or greater in the past week.'}


In [18]:
print(response.final_answer["content"])

There have been 32 earthquakes with a magnitude of 5 or greater in the past week.


In [19]:
response = agent.invoke({"question": "Can MegaQuakes really happen? Like a magnitude 10 or larger?"})

03/05/24 16:26:08 INFO Thought: The user is asking about the possibility of a magnitude 10 or larger earthquake, also known as a 'MegaQuake'. This is a question about the theoretical limits of earthquake magnitude, which is determined by the size of the fault on which it occurs. The largest earthquake ever recorded was a magnitude 9.5 in Chile in 1960. A magnitude 10 earthquake would require a fault rupture along the entire length of the earth's largest fault, the San Andreas, which is highly unlikely. I will provide this information to the user.
03/05/24 16:26:08 INFO Tool: Final Answer
03/05/24 16:26:08 INFO Tool input: {'content': "While theoretically possible, a magnitude 10 or larger earthquake, also known as a 'MegaQuake', is highly unlikely. The magnitude of an earthquake is determined by the size of the fault on which it occurs. The largest earthquake ever recorded was a magnitude 9.5 in Chile in 1960. A magnitude 10 earthquake would require a fault rupture along the entire len

In [20]:
print(response.final_answer["content"])

While theoretically possible, a magnitude 10 or larger earthquake, also known as a 'MegaQuake', is highly unlikely. The magnitude of an earthquake is determined by the size of the fault on which it occurs. The largest earthquake ever recorded was a magnitude 9.5 in Chile in 1960. A magnitude 10 earthquake would require a fault rupture along the entire length of the earth's largest fault, the San Andreas, which is highly unlikely.


In [21]:
response = agent.invoke({"question": "Query 10 earthquakes that occurred yesterday and have a magnitude >= 5."})

03/05/24 16:26:29 INFO Thought: To answer the user's question, I need to find earthquakes that occurred yesterday with a magnitude of 5 or greater. I will use the 'Query' tool to find this information.
03/05/24 16:26:29 INFO Tool: Query
03/05/24 16:26:29 INFO Tool input: {'start_time': '2024-05-02T00:00:00', 'end_time': '2024-05-02T23:59:59', 'min_magnitude': 5, 'limit': 10}
03/05/24 16:26:30 INFO Tool response: {'type': 'FeatureCollection', 'metadata': {'generated': 1714746390000, 'url': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2024-05-02T00%3A00%3A00&endtime=2024-05-02T23%3A59%3A59&limit=10&mindepth=-100&maxdepth=1000&minmagnitude=5', 'title': 'USGS Earthquakes', 'status': 200, 'api': '1.14.1', 'limit': 10, 'offset': 1, 'count': 9}, 'features': [{'type': 'Feature', 'properties': {'mag': 5.1, 'place': 'south of the Kermadec Islands', 'time': 1714693022188, 'updated': 1714694206040, 'tz': None, 'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/

In [22]:
print(response.final_answer["content"])

Yesterday, there were 9 earthquakes with a magnitude of 5 or greater. The locations of these earthquakes varied, with some occurring in the South Sandwich Islands region, south of the Kermadec Islands, and near Nikolski, Alaska. The magnitudes ranged from 5.0 to 5.5. None of these earthquakes reached a magnitude of 10 or greater.


## 4. Service-based AI Agents

In [23]:
import pandas as pd
from sklearn.metrics import accuracy_score
from language_models.agents.chain import AgentChain

In [28]:
df = pd.read_csv("./data/tweets.csv.gz", compression="gzip", encoding="latin-1", names=["sentiment", "id", "date", "query", "user", "tweet"])
df = df.iloc[:8]
df.head()

,sentiment,id,date,query,user,tweet
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [37]:
system_prompt = """Take the following tweet and determine the sentiment of the review. 

Respond with 1 (positive) or 0 (negative).

If you don't receive a tweet, respond with -1.
"""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=128,
    float=0.0,
)

class Output(BaseModel):
    sentiment: int = Field(description="The sentiment of the tweet.")

agent = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt="Tweet content: {tweet}",
    task_prompt_variables=["tweet"],
    tools=None,
    output_format=Output,
    iterations=5,
)

In [38]:
def classify_sentiment(tweet: str) -> int:
    response = agent.invoke({'tweet': tweet})
    return response.final_answer['sentiment']

In [39]:

df["prediction"] = [classify_sentiment(tweet) for tweet in df.tweet]

03/05/24 16:32:02 INFO Thought: The tweet seems to express a mild disappointment but also suggests a solution in a friendly tone. It's not overly negative, but it's not exactly positive either. It's more of a neutral sentiment with a slight lean towards negative due to the expression of disappointment.
03/05/24 16:32:02 INFO Tool: Final Answer
03/05/24 16:32:02 INFO Tool input: {'sentiment': 0}
03/05/24 16:32:08 INFO Thought: The tweet expresses disappointment and sadness, which indicates a negative sentiment.
03/05/24 16:32:08 INFO Tool: Final Answer
03/05/24 16:32:08 INFO Tool input: {'sentiment': 0}
03/05/24 16:32:18 INFO Thought: The tweet seems to be neutral. The user is talking about a game where they managed to save 50% of the balls, but the rest went out of bounds. There's no clear positive or negative sentiment.
03/05/24 16:32:18 INFO Tool: Final Answer
03/05/24 16:32:18 INFO Tool input: {'sentiment': 1}
03/05/24 16:32:24 INFO Thought: The tweet seems to express discomfort and

In [40]:
print(f"Accuracy: {accuracy_score(df.sentiment, df.prediction)}")

Accuracy: 0.75
